In [7]:
import os
import numpy as np
from PIL import Image
from sklearn.metrics import precision_score, recall_score, f1_score, jaccard_score
import cv2
import re
# import os
import glob
# import re
# import numpy as np
# from sklearn.metrics import jaccard_score, precision_score, recall_score, f1_score
import tifffile as tiff
# import os
# import numpy as np
# import glob
# from sklearn.metrics import jaccard_score, precision_score, recall_score, f1_score
# import rasterio

def load_image(filepath):
    # Load the image using PIL
    image = np.array(Image.open(filepath).convert('L'))
    
    # Apply Otsu's thresholding
    t, binary_image = cv2.threshold(image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    
    # Convert to binary (0 and 1)
    binary_image = (binary_image > 0).astype(np.uint8)
    return binary_image

def dice_coefficient(y_true, y_pred):
    intersection = np.sum(y_true * y_pred)
    return (2. * intersection) / (np.sum(y_true) + np.sum(y_pred))

def pixel_accuracy(y_true, y_pred):
    return np.mean(y_true == y_pred)

def specificity(y_true, y_pred):
    tn = np.sum((y_true == 0) & (y_pred == 0))
    fp = np.sum((y_true == 0) & (y_pred == 1))
    return tn / (tn + fp)

# Paths to the folders containing the predictions and ground truths


In [2]:
# gt_folder = './GEE_Masks/GEE_resized/'
# pred_folder = './GEE_Output/Adversarial'

# # List all files in the prediction folder
# # pred_files = [f for f in os.listdir(pred_folder) if f.endswith('.tif')]

# # Initialize metrics



In [3]:
# # Lists to store metrics
# dice_scores = []
# iou_scores = []
# pixel_accuracies = []
# precisions = []
# recalls = []
# f1_scores = []
# specificities = []

# for i in range(0, 1263):  # Adjust the range based on the number of files
#     gt_filename = f'NDWI_Mask_{i}_resized.tif'
#     pred_filename = f'dense_Predicted_Image_{i}.tif'
    
#     gt_path = os.path.join(gt_folder, gt_filename)
#     pred_path = os.path.join(pred_folder, pred_filename)
    
#     # Load the prediction and ground truth images
#     pred_image = load_image(pred_path)
#     gt_image = load_image(gt_path)
    
#     # Flatten the images
#     pred_flat = pred_image.flatten()
#     gt_flat = gt_image.flatten()
    
#     # Calculate metrics
#     dice = dice_coefficient(gt_flat, pred_flat)
#     iou = jaccard_score(gt_flat, pred_flat, average='macro')
#     accuracy = pixel_accuracy(gt_flat, pred_flat)
#     precision = precision_score(gt_flat, pred_flat, average='macro', zero_division=0)
#     recall = recall_score(gt_flat, pred_flat, average='macro')
#     f1 = f1_score(gt_flat, pred_flat, average='macro')
#     spec = specificity(gt_flat, pred_flat)
    
#     # Store metrics
#     dice_scores.append(dice)
#     iou_scores.append(iou)
#     pixel_accuracies.append(accuracy)
#     precisions.append(precision)
#     recalls.append(recall)
#     f1_scores.append(f1)
#     specificities.append(spec)
    
#     # Print metrics for the current image
#     print(f"Metrics for {gt_filename} and {pred_filename}:")
#     print(f"  Dice Coefficient: {dice:.4f}")
#     print(f"  IoU: {iou:.4f}")
#     print(f"  Pixel Accuracy: {accuracy:.4f}")
#     print(f"  Precision: {precision:.4f}")
#     print(f"  Recall: {recall:.4f}")
#     print(f"  F1 Score: {f1:.4f}")
#     print(f"  Specificity: {spec:.4f}")
#     print()

# # Calculate and print average metrics
# average_dice = np.mean(dice_scores)
# average_iou = np.mean(iou_scores)
# average_accuracy = np.mean(pixel_accuracies)
# average_precision = np.mean(precisions)
# average_recall = np.mean(recalls)
# average_f1 = np.mean(f1_scores)
# average_specificity = np.mean(specificities)

# print("Average metrics:")
# print(f"  Average Dice Coefficient: {average_dice:.4f}")
# print(f"  Average IoU: {average_iou:.4f}")
# print(f"  Average Pixel Accuracy: {average_accuracy:.4f}")
# print(f"  Average Precision: {average_precision:.4f}")
# print(f"  Average Recall: {average_recall:.4f}")
# print(f"  Average F1 Score: {average_f1:.4f}")
# print(f"  Average Specificity: {average_specificity:.4f}")

# # Model's overall accuracy
# print(f"Model's Overall Accuracy: {average_accuracy:.4f}")

In [4]:
# # Print average metrics
# print("Average Metrics:")
# print(f"  Average Dice Coefficient: {np.mean(dice_scores):.4f}")
# print(f"  Average IoU: {np.mean(iou_scores):.4f}")
# print(f"  Average Pixel Accuracy: {np.mean(pixel_accuracies):.4f}")
# print(f"  Average Precision: {np.mean(precisions):.4f}")
# print(f"  Average Recall: {np.mean(recalls):.4f}")
# print(f"  Average F1 Score: {np.mean(f1_scores):.4f}")
# print(f"  Average Specificity: {np.mean(specificities):.4f}")

In [8]:
def dice_coefficient(y_true, y_pred, smooth=1):
    y_true_f = y_true.flatten()
    y_pred_f = y_pred.flatten()
    intersection = np.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (np.sum(y_true_f) + np.sum(y_pred_f) + smooth)

def pixel_accuracy(y_true, y_pred):
    return np.mean(y_true == y_pred)

def specificity(y_true, y_pred):
    tn = np.sum((y_true == 0) & (y_pred == 0))
    fp = np.sum((y_true == 0) & (y_pred == 1))
    return tn / (tn + fp)


# Directories for ground truth and prediction masks
gt_folder = './GEE_Masks/GEE_resized/'
pred_folder = './GEE_Output/Adversarial'

# Initialize metric lists
dice_scores = []
iou_scores = []
pixel_accuracies = []
precisions = []
recalls = []
f1_scores = []
specificities = []

# Get all prediction files
pred_files = glob.glob(os.path.join(pred_folder, 'dense_*.tif'))

for pred_file in pred_files:
    # Extract the number 'i' from the prediction filename
    match = re.search(r'dense_(\d+).tif', os.path.basename(pred_file))
    if match:
        i = match.group(1)
        gt_filename = f'NDWI_Mask_{i}_resized.tif'
        gt_path = os.path.join(gt_folder, gt_filename)
        
        # Check if the ground truth file exists
        if os.path.exists(gt_path):
            # Load the prediction and ground truth images
            pred_image = load_image(pred_file)
            gt_image = load_image(gt_path)
            print(np.max(pred_image))
            print(np.min(pred_image))
            # Flatten the images
            pred_flat = pred_image.flatten()
            gt_flat = gt_image.flatten()
            
            # Calculate metrics
            dice = dice_coefficient(gt_flat, pred_flat)
            iou = jaccard_score(gt_flat, pred_flat, average='macro')
            accuracy = pixel_accuracy(gt_flat, pred_flat)
            precision = precision_score(gt_flat, pred_flat, average='macro', zero_division=0)
            recall = recall_score(gt_flat, pred_flat, average='macro')
            f1 = f1_score(gt_flat, pred_flat, average='macro')
            spec = specificity(gt_flat, pred_flat)
            
            # Store metrics
            dice_scores.append(dice)
            iou_scores.append(iou)
            pixel_accuracies.append(accuracy)
            precisions.append(precision)
            recalls.append(recall)
            f1_scores.append(f1)
            specificities.append(spec)
            
            # Print metrics for the current image
            print(f"Metrics for {gt_filename} and {pred_file}:")
            print(f"  Dice Coefficient: {dice:.4f}")
            print(f"  IoU: {iou:.4f}")
            print(f"  Pixel Accuracy: {accuracy:.4f}")
            print(f"  Precision: {precision:.4f}")
            print(f"  Recall: {recall:.4f}")
            print(f"  F1 Score: {f1:.4f}")
            print(f"  Specificity: {spec:.4f}")
            print()
        else:
            print(f"Ground truth file {gt_path} not found for prediction file {pred_file}.")

# Calculate and print average metrics
average_dice = np.mean(dice_scores)
average_iou = np.mean(iou_scores)
average_accuracy = np.mean(pixel_accuracies)
average_precision = np.mean(precisions)
average_recall = np.mean(recalls)
average_f1 = np.mean(f1_scores)
average_specificity = np.mean(specificities)

print("Average Metrics:")
print(f"  Dice Coefficient: {average_dice:.4f}")
print(f"  IoU: {average_iou:.4f}")
print(f"  Pixel Accuracy: {average_accuracy:.4f}")
print(f"  Precision: {average_precision:.4f}")
print(f"  Recall: {average_recall:.4f}")
print(f"  F1 Score: {average_f1:.4f}")
print(f"  Specificity: {average_specificity:.4f}")

1
0
Metrics for NDWI_Mask_1_resized.tif and ./GEE_Output/Adversarial\dense_1.tif:
  Dice Coefficient: 0.9677
  IoU: 0.8800
  Pixel Accuracy: 0.9515
  Precision: 0.9661
  Recall: 0.9126
  F1 Score: 0.9352
  Specificity: 0.8275

1
0
Metrics for NDWI_Mask_10_resized.tif and ./GEE_Output/Adversarial\dense_10.tif:
  Dice Coefficient: 0.7072
  IoU: 0.6390
  Pixel Accuracy: 0.7969
  Precision: 0.8037
  Recall: 0.7664
  F1 Score: 0.7759
  Specificity: 0.9114

1
0
Metrics for NDWI_Mask_100_resized.tif and ./GEE_Output/Adversarial\dense_100.tif:
  Dice Coefficient: 0.8511
  IoU: 0.8652
  Pixel Accuracy: 0.9899
  Precision: 0.8715
  Recall: 0.9928
  F1 Score: 0.9230
  Specificity: 0.9897

1
0
Metrics for NDWI_Mask_1000_resized.tif and ./GEE_Output/Adversarial\dense_1000.tif:
  Dice Coefficient: 0.0058
  IoU: 0.4737
  Pixel Accuracy: 0.9446
  Precision: 0.4998
  Recall: 0.4967
  F1 Score: 0.4886
  Specificity: 0.9478

1
0
Metrics for NDWI_Mask_1001_resized.tif and ./GEE_Output/Adversarial\dense_10

c:\Users\ADMIN\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Metrics for NDWI_Mask_1077_resized.tif and ./GEE_Output/Adversarial\dense_1077.tif:
  Dice Coefficient: 0.0081
  IoU: 0.4998
  Pixel Accuracy: 0.9995
  Precision: 0.5000
  Recall: 0.4998
  F1 Score: 0.4999
  Specificity: 0.9995

1
0
Metrics for NDWI_Mask_108_resized.tif and ./GEE_Output/Adversarial\dense_108.tif:
  Dice Coefficient: 0.9688
  IoU: 0.9550
  Pixel Accuracy: 0.9797
  Precision: 0.9723
  Recall: 0.9819
  F1 Score: 0.9769
  Specificity: 0.9759

1
0


c:\Users\ADMIN\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\ADMIN\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Metrics for NDWI_Mask_1081_resized.tif and ./GEE_Output/Adversarial\dense_1081.tif:
  Dice Coefficient: 0.0400
  IoU: 0.5000
  Pixel Accuracy: 0.9999
  Precision: 0.5000
  Recall: 0.5000
  F1 Score: 0.5000
  Specificity: 0.9999

1
0
Metrics for NDWI_Mask_1082_resized.tif and ./GEE_Output/Adversarial\dense_1082.tif:
  Dice Coefficient: 0.3333
  IoU: 0.5000
  Pixel Accuracy: 1.0000
  Precision: 0.5000
  Recall: 0.5000
  F1 Score: 0.5000
  Specificity: 1.0000

1
0
Metrics for NDWI_Mask_1086_resized.tif and ./GEE_Output/Adversarial\dense_1086.tif:
  Dice Coefficient: 0.0317
  IoU: 0.5065
  Pixel Accuracy: 0.9992
  Precision: 0.5097
  Recall: 0.5218
  F1 Score: 0.5134
  Specificity: 0.9994

1
0
Metrics for NDWI_Mask_1087_resized.tif and ./GEE_Output/Adversarial\dense_1087.tif:
  Dice Coefficient: 0.1964
  IoU: 0.5498
  Pixel Accuracy: 0.9909
  Precision: 0.6191
  Recall: 0.5803
  F1 Score: 0.5958
  Specificity: 0.9965

1
0
Metrics for NDWI_Mask_1089_resized.tif and ./GEE_Output/Adversarial\

...

Metrics for NDWI_Mask_90_resized.tif and ./GEE_Output/Adversarial\dense_90.tif:
  Dice Coefficient: 0.0057
  IoU: 0.4997
  Pixel Accuracy: 0.9993
  Precision: 0.5000
  Recall: 0.4997
  F1 Score: 0.4998
  Specificity: 0.9993

1
0
Metrics for NDWI_Mask_900_resized.tif and ./GEE_Output/Adversarial\dense_900.tif:
  Dice Coefficient: 0.8288
  IoU: 0.7103
  Pixel Accuracy: 0.8306
  Precision: 0.8325
  Recall: 0.8332
  F1 Score: 0.8306
  Specificity: 0.7917

1
0
Metrics for NDWI_Mask_902_resized.tif and ./GEE_Output/Adversarial\dense_902.tif:
  Dice Coefficient: 0.7346
  IoU: 0.7078
  Pixel Accuracy: 0.8658
  Precision: 0.8223
  Recall: 0.8224
  F1 Score: 0.8224
  Specificity: 0.9101

1
0
Metrics for NDWI_Mask_904_resized.tif and ./GEE_Output/Adversarial\dense_904.tif:
  Dice Coefficient: 0.2671
  IoU: 0.5531
  Pixel Accuracy: 0.9524
  Precision: 0.6004
  Recall: 0.6566
  F1 Score: 0.6213
  Specificity: 0.9681

1
0
Metrics for NDWI_Mask_906_resized.tif and ./GEE_Output/Adversarial\dense_906.t

c:\Users\ADMIN\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\ADMIN\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Metrics for NDWI_Mask_95_resized.tif and ./GEE_Output/Adversarial\dense_95.tif:
  Dice Coefficient: 0.0021
  IoU: 0.4991
  Pixel Accuracy: 0.9982
  Precision: 0.5000
  Recall: 0.4991
  F1 Score: 0.4995
  Specificity: 0.9982

1
0
Metrics for NDWI_Mask_952_resized.tif and ./GEE_Output/Adversarial\dense_952.tif:
  Dice Coefficient: 0.1667
  IoU: 0.5000
  Pixel Accuracy: 1.0000
  Precision: 0.5000
  Recall: 0.5000
  F1 Score: 0.5000
  Specificity: 1.0000

1
0


c:\Users\ADMIN\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Metrics for NDWI_Mask_955_resized.tif and ./GEE_Output/Adversarial\dense_955.tif:
  Dice Coefficient: 0.0189
  IoU: 0.4999
  Pixel Accuracy: 0.9998
  Precision: 0.5000
  Recall: 0.4999
  F1 Score: 0.5000
  Specificity: 0.9998

1
0
Metrics for NDWI_Mask_956_resized.tif and ./GEE_Output/Adversarial\dense_956.tif:
  Dice Coefficient: 0.9047
  IoU: 0.9081
  Pixel Accuracy: 0.9907
  Precision: 0.9135
  Recall: 0.9944
  F1 Score: 0.9499
  Specificity: 0.9903

1
0
Metrics for NDWI_Mask_957_resized.tif and ./GEE_Output/Adversarial\dense_957.tif:
  Dice Coefficient: 0.0056
  IoU: 0.4997
  Pixel Accuracy: 0.9993
  Precision: 0.4997
  Recall: 0.5000
  F1 Score: 0.4998
  Specificity: 0.9999

1
0
Metrics for NDWI_Mask_958_resized.tif and ./GEE_Output/Adversarial\dense_958.tif:
  Dice Coefficient: 0.0115
  IoU: 0.4998
  Pixel Accuracy: 0.9997
  Precision: 0.4999
  Recall: 0.5000
  F1 Score: 0.4999
  Specificity: 1.0000

1
0
Metrics for NDWI_Mask_959_resized.tif and ./GEE_Output/Adversarial\dense_959

In [9]:
print("Average metrics:")
print(f"  Average Dice Coefficient: {average_dice:.4f}")
print(f"  Average IoU: {average_iou:.4f}")
print(f"  Average Pixel Accuracy: {average_accuracy:.4f}")
print(f"  Average Precision: {average_precision:.4f}")
print(f"  Average Recall: {average_recall:.4f}")
print(f"  Average F1 Score: {average_f1:.4f}")
print(f"  Average Specificity: {average_specificity:.4f}")

# Model's overall accuracy
print(f"Model's Overall Accuracy: {average_accuracy:.4f}")

Average metrics:
  Average Dice Coefficient: 0.6784
  Average IoU: 0.7527
  Average Pixel Accuracy: 0.9357
  Average Precision: 0.8111
  Average Recall: 0.8437
  Average F1 Score: 0.8133
  Average Specificity: 0.9345
Model's Overall Accuracy: 0.9357
